# Tutorial: Retrieval of documents from a corpus using Neural Information Retrieval (IR)

In this tutorial you'll learn how to use a popular Neural IR system called DPR [Karpukhin2020].


## Preparing a Colab Environment to run this tutorial ##

Make sure to "Enable GPU Runtime" -> make a URL with a page with screenshots on how to do this.

## Installing PrimeQA

First, we need to include the required modules.


In [ ]:
%%bash

pip install --upgrade pip
pip install primeqa

## Pre-process your document collection here to be ready to be stored in your Neural Search Index.
In this step we download a publicly available .csv file from a Google Drive location and save it as .tsv.

In [1]:
# save your input document as a .tsv
import pandas as pd
url='https://drive.google.com/file/d/1LULJRPgN_hfuI2kG-wH4FUwXCCdDh9zh/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.to_csv('input.tsv', sep='\t')

## Step 1: Init -- Initialize your model. In PrimeQA for searching through your corpus, we use a class called SearchableCorpus.

For DPR, you need to point to a question and context encoder models available via the HuggingFace model hub.

In [2]:
from primeqa.components.retriever.searchable_corpus import SearchableCorpus
collection = SearchableCorpus(model_name="PrimeQA/XOR-TyDi_monolingual_DPR_ctx_encoder", 
                              query_encoder_model_name_or_path="PrimeQA/XOR-TyDi_monolingual_DPR_qry_encoder", 
                              batch_size=64, top_k=10)

{"time":"2023-06-14 14:01:11,971", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss with AVX2 support."}
{"time":"2023-06-14 14:01:12,417", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss with AVX2 support."}


## Step 2: Add your documents into the searchable corpus through PrimeQA's built-in pre-processor.

PrimeQA has a built-in class called DocumentCollection which pre-processes input.tsv to match the following format as needed by DPR:

`id \t text \t title_of_document`

Note: since DPR is based on an encoder language model the typical sequence length is 512 max sub-word tokens. So please make sure your documents are split into text length of ~220 words.

In [3]:
from primeqa.ir.util.corpus_reader import DocumentCollection
doc_collection = DocumentCollection("input.tsv")

collection.add_documents(doc_collection.get_processed_collection())

75it [00:00, 42561.60it/s]


75


75it [00:00, 65169.42it/s]


75
{"time":"2023-06-14 14:01:53,070", "name": "primeqa.ir.dense.dpr_top.dpr.index_simple_corpus", "level": "INFO", "message": "wrote passages_1_of_1.json.gz.records in 0 seconds"}
{"time":"2023-06-14 14:01:53,071", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "building index, reading data from dpr_index_dir/passages_1_of_1.json.gz.records, writing to dpr_index_dir/index_1_of_1.faiss"}
{"time":"2023-06-14 14:01:53,121", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "processed 0 passages"}
{"time":"2023-06-14 14:01:53,124", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "calling index.add with 75 vectors"}
{"time":"2023-06-14 14:01:53,127", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "processed 75 passages"}
{"time":"2023-06-14 14:01:53,127", "name": "primeqa.ir.dense.dpr_top.dpr.faiss_index", "level": "INFO", "message": "finished building index, writi

Some weights of the model checkpoint at PrimeQA/XOR-TyDi_monolingual_DPR_ctx_encoder were not used when initializing DPRQuestionEncoder: ['ctx_encoder.bert_model.encoder.layer.8.attention.output.dense.weight', 'ctx_encoder.bert_model.encoder.layer.1.attention.self.key.weight', 'ctx_encoder.bert_model.encoder.layer.3.attention.self.query.weight', 'ctx_encoder.bert_model.encoder.layer.0.attention.output.dense.bias', 'ctx_encoder.bert_model.encoder.layer.6.intermediate.dense.bias', 'ctx_encoder.bert_model.encoder.layer.4.attention.self.key.weight', 'ctx_encoder.bert_model.encoder.layer.2.attention.output.LayerNorm.bias', 'ctx_encoder.bert_model.encoder.layer.5.attention.self.query.weight', 'ctx_encoder.bert_model.encoder.layer.7.attention.self.value.weight', 'ctx_encoder.bert_model.encoder.layer.7.attention.self.query.bias', 'ctx_encoder.bert_model.encoder.layer.10.output.dense.weight', 'ctx_encoder.bert_model.encoder.layer.2.attention.self.value.bias', 'ctx_encoder.bert_model.encoder.lay

Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at PrimeQA/XOR-TyDi_monolingual_DPR_ctx_encoder and are newly initialized: ['bert_model.encoder.layer.1.intermediate.dense.weight', 'bert_model.encoder.layer.0.intermediate.dense.bias', 'bert_model.encoder.layer.2.intermediate.dense.bias', 'bert_model.encoder.layer.7.attention.self.query.bias', 'bert_model.encoder.layer.10.attention.output.dense.weight', 'bert_model.encoder.layer.5.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.self.value.weight', 'bert_model.encoder.layer.2.attention.self.query.weight', 'bert_model.encoder.layer.3.output.dense.bias', 'bert_model.encoder.layer.4.attention.self.value.weight', 'bert_model.encoder.layer.11.attention.self.key.bias', 'bert_model.encoder.layer.11.intermediate.dense.bias', 'bert_model.encoder.layer.7.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.2.output.dense.weight', 'bert_model.encoder.layer.5.intermediate.dense.weight

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRContextEncoderTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.


{"time":"2023-06-14 14:01:54,550", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Using sharded faiss, reading shards from dpr_index_dir"}
{"time":"2023-06-14 14:01:54,551", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Reading passages_1_of_1.json.gz.records"}
{"time":"2023-06-14 14:01:54,552", "name": "primeqa.ir.dense.dpr_top.dpr.searcher", "level": "INFO", "message": "Using sharded faiss with 1 shards."}


## Step 3: Search -- start asking questions.

Your queries can be a list. You can also retrieve the scores of retrieved documents.

In [4]:
queries = ['When was Idaho split in two?' , 'Who was Danny Nozel']
retrieved_doc_ids, passages = collection.search(queries)
#res, scores = collection.search2(queries)
import json
print(json.dumps(passages, indent = 4))

[
    {
        "titles": [
            "\"Akira Kurosawa\"",
            "\"Alfred Nobel\"",
            "\"Alkali metal\"",
            "\"Alkali metal\"",
            "Anatomy",
            "Anime",
            "Anatomy",
            "\"Albert Einstein\"",
            "\"Albert Einstein\"",
            "\"Alkali metal\""
        ],
        "texts": [
            "Akira Kurosawa Akira Kurosawa (, \"Kurosawa Akira\"; March 23, 1910 \u2013 September 6, 1998) was a Japanese film director and screenwriter, who directed 30 films in a career spanning 57 years. He is regarded as one of the most important and influential filmmakers in the history of cinema. Kurosawa entered the Japanese film industry in 1936, following a brief stint as a painter. After years of working on numerous films as an assistant director and scriptwriter, he made his debut as a director during World War II with the popular action film \"Sanshiro Sugata\" (a.k.a. \"Judo Saga\"). After the war,",
            "was adopte